<h1>This is a notebook to implement the Final Assignment of IBM Data Science course on Coursera.</h1>

My name is Rafael Fensterseifer and I am from Porto Alegre, Rio Grande do Sul state, Brasil

<h1>
Introduction: Business Problem
</h1>

It is almost a concensus that in "universitary cities" life is more agitated. There are lots of bars, parties, restaurants and other types of places releated to that.

Each region in a city has a profile of public venues.

The question that this research will try to answer is: is there any correlation between regions venues profile and the presence (or not) of universities in that region?

<h1>1 - First of all, get our data!</h1>

To do this analisys, the city of Toronto will be used for. To start, we need to define the regions of study. For it, we will use the postal code segmentation of the city as base.

To start our data collection, we will search for all universities in Toronto and define what is its postal codes. So, we can define the total number of universities on each postal code region. For each region, we will analise the venues to define a vanues profile.

<h3> 1.1 - List of Postal Codes of Toronto (Ontario/Canada) and its latitude and longitude coordinates</h3>

In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', header=0)[0]
df.rename(columns={'Postal Code':'PostalCode'}, inplace=True)
df.replace('Not assigned', np.nan, inplace=True)
df.dropna(subset=['Borough'], axis=0, inplace=True)
df.head()

,PostalCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


<h4>
    In the next code cell, data was loaded from project resources.<br/>
    The data is a file containing the latitude and longitude coordinates of each postal code area of Toronto. Dataframe <i>dfPostalCodes</i> is join result of that data and dataframe <i>df</i>.
</h4>

In [4]:
# The code was removed by Watson Studio for sharing.

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [5]:
dfPostalCodes = df.merge(df_data_1, on='PostalCode', how='left')
dfPostalCodes.head()


,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


<h3>1.2 - List of Toronto Universities and its latitude and longitude coordinates</h3>

<h4>
    Scrapping the web, I found a page with a list of all universities in Toronto.<br/>
    So I read that and loaded the data into a new dataframe.
</h4>

In [6]:
url = 'http://www.unipage.net/en/universities/toronto'

import requests

r = requests.get(url)
dfUniversities = pd.read_html(r.text)[0]
dfUniversities = dfUniversities[['Title']]
dfUniversities

,Title
0,University of Toronto
1,York University
2,Ryerson University
3,Humber College
4,Centennial College
5,George Brown College
6,OCAD University
7,Michener Institute for Applied Health Sciences
8,Canadian Memorial Chiropractic College
9,Tyndale College & Seminary


In [7]:
dfUniversities.shape

(22, 1)

<h4>
    Using Nominatim, I tried to locate all universities coordinates (latitude and longitude).
</h4>

In [8]:
# The code was removed by Watson Studio for sharing.

In [9]:
# !pip install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

In [10]:
geolocator = Nominatim(user_agent="foursquare_agent")

In [11]:
def f(row):
    location = geolocator.geocode('{}, Toronto, ON, CA'.format(row.Title))
    if location != None:
        return location.latitude, location.longitude

dfUniversities['coords'] = dfUniversities.apply(f, axis=1)


In [12]:
dfUniversities[['lat','lon']] = pd.DataFrame(dfUniversities['coords'].tolist(), index=dfUniversities.index)
#dfUniversities['lat'], dfUniversities['lon'] = dfUniversities['coords'].apply(lambda c: c if c != None else (np.nan,np.nan))

In [13]:
dfUniversities.drop(columns='coords', inplace=True)
dfUniversities

,Title,lat,lon
0,University of Toronto,43.663462,-79.397760
1,York University,43.779242,-79.483559
2,Ryerson University,43.658469,-79.378993
3,Humber College,43.728512,-79.606250
4,Centennial College,43.785792,-79.227810
5,George Brown College,43.661689,-79.391492
6,OCAD University,43.649334,-79.388887
7,Michener Institute for Applied Health Sciences,NaN,NaN
8,Canadian Memorial Chiropractic College,NaN,NaN
9,Tyndale College & Seminary,NaN,NaN


<h4>
    As you can see above, <i>Nominatin</i> does not found some Universities.<br/>
    <br/>
    Because it is a fundamental data to my analysis, I put those missing values manually, having searched for each one in Google Maps.
</h4>

In [14]:
dfUniversities.iloc[7, [1, 2]] = 43.656068, -79.390157
dfUniversities.iloc[8, [1, 2]] = 43.804010, -79.371692
dfUniversities.iloc[9, [1, 2]] = 43.797215, -79.392058
dfUniversities.iloc[10, [1, 2]] = 43.664140, -79.389595
dfUniversities.iloc[13, [1, 2]] = 43.696793, -79.395161
dfUniversities.iloc[14, [1, 2]] = 43.665362, -79.395217
dfUniversities.iloc[15, [1, 2]] = 43.707839, -79.394081
dfUniversities.iloc[16, [1, 2]] = 43.707852, -79.394303
dfUniversities.iloc[17, [1, 2]] = 43.674188, -79.388516
dfUniversities.iloc[18, [1, 2]] = 43.709327, -79.394502
dfUniversities.iloc[19, [1, 2]] = 43.661566, -79.396597
dfUniversities.iloc[20, [1, 2]] = 43.667066, -79.389729
dfUniversities.iloc[21, [1, 2]] = 43.664792, -79.394359

In [15]:
dfUniversities

,Title,lat,lon
0,University of Toronto,43.663462,-79.397760
1,York University,43.779242,-79.483559
2,Ryerson University,43.658469,-79.378993
3,Humber College,43.728512,-79.606250
4,Centennial College,43.785792,-79.227810
5,George Brown College,43.661689,-79.391492
6,OCAD University,43.649334,-79.388887
7,Michener Institute for Applied Health Sciences,43.656068,-79.390157
8,Canadian Memorial Chiropractic College,43.804010,-79.371692
9,Tyndale College & Seminary,43.797215,-79.392058


<h4>
    Now we can see all locations on a map of Toronto.
</h4>

In [16]:
#!conda install -c conda-forge folium=0.5.0 --yes
!pip install folium
import folium # plotting library

     |████████████████████████████████| 102kB 9.6MB/s eta 0:00:01


In [17]:
dfUniversitiesClean = dfUniversities.dropna(subset=['lat', 'lon'], axis=0)

In [18]:
latToronto = 43.651070
lonToronto = -79.347015
zoom = 11

venues_map = folium.Map(location=[latToronto, lonToronto], zoom_start=zoom) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.CircleMarker(
    [latToronto, lonToronto],
    radius=10,
    color='red',
    popup='Toronto',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

for lat, lon, label in zip(dfUniversitiesClean.lat, dfUniversitiesClean.lon, dfUniversitiesClean.Title):
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)
    
venues_map

<h4>
    Using Geocoder (a system that resolves addresses in CA and US), we can convert the universities coordinates into postal code areas.<br/>
</h4>

In [19]:
!pip install geocoder
import geocoder

     |████████████████████████████████| 102kB 8.3MB/s eta 0:00:01


In [20]:
def getPostalCode(lat, lon):
    neigh = geocoder.geolytica('{}, {}'.format(lat, lon))
    return neigh.json['postal'][:3]

In [21]:
dfUniversitiesClean['PostalCode'] = dfUniversitiesClean.apply(lambda row : getPostalCode(row.lat, row.lon), axis=1)

In [22]:
dfUniversitiesClean

,Title,lat,lon,PostalCode
0,University of Toronto,43.663462,-79.397760,M5S
1,York University,43.779242,-79.483559,M3J
2,Ryerson University,43.658469,-79.378993,M5B
3,Humber College,43.728512,-79.606250,M9W
4,Centennial College,43.785792,-79.227810,M1G
5,George Brown College,43.661689,-79.391492,M7A
6,OCAD University,43.649334,-79.388887,M5V
7,Michener Institute for Applied Health Sciences,43.656068,-79.390157,M5T
8,Canadian Memorial Chiropractic College,43.804010,-79.371692,M2H
9,Tyndale College & Seminary,43.797215,-79.392058,M2M


<h4>
    Now, we can calculate the number of universities per postal code area.
</h4>

In [23]:
dfPostalCodesUniversitiesCount = dfUniversitiesClean[['PostalCode', 'Title']].groupby(['PostalCode']).count().rename(columns={'Title':'Universities'})
dfPostalCodesUniversitiesCount

,Universities
PostalCode,
M1G,1
M2H,1
M2J,1
M2M,1
M3J,1
M4P,3
M4S,1
M4W,1
M5B,1


<h4>
    Finally, we can merge the universities count onto postal codes dataframe.
</h4>

In [24]:
dfPostalCodesUniversities = dfPostalCodes.join(dfPostalCodesUniversitiesCount, on='PostalCode').replace(np.nan, 0)
#dfPostalCodesUniversities['Universities'] = dfPostalCodesUniversities['Universities'].astype(int)
dfPostalCodesUniversities[dfPostalCodesUniversities['Universities'] != 0]

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Universities
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,1.0
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,1.0
22,M1G,Scarborough,Woburn,43.770992,-79.216917,1.0
27,M2H,North York,Hillcrest Village,43.803762,-79.363452,1.0
33,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,1.0
34,M3J,North York,"Northwood Park, York University",43.767980,-79.487262,1.0
52,M2M,North York,"Willowdale, Newtonbrook",43.789053,-79.408493,1.0
67,M4P,Central Toronto,Davisville North,43.712751,-79.390197,3.0
79,M4S,Central Toronto,Davisville,43.704324,-79.388790,1.0
80,M5S,Downtown Toronto,"University of Toronto, Harbord",43.662696,-79.400049,7.0


In [25]:
dfPostalCodesUniversities.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Universities
0,M3A,North York,Parkwoods,43.753259,-79.329656,0.0
1,M4A,North York,Victoria Village,43.725882,-79.315572,0.0
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0.0
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,0.0
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,1.0


<h3>
    1.3 - Using Foursquare API to explore the venues of each Postal Code area
</h3>

In [26]:
# The code was removed by Watson Studio for sharing.

In [27]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
        
        print('{} - {}'.format(name, len(results)))

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['PostalCode', 
                  'Latitude', 
                  'Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [28]:
nearby_venues = getNearbyVenues(dfPostalCodesUniversities['PostalCode'], 
                                dfPostalCodesUniversities['Latitude'], 
                                dfPostalCodesUniversities['Longitude'])

M3A - 3
M4A - 5
M5A - 30
M6A - 9
M7A - 30
M9A - 0
M1B - 1
M3B - 5
M4B - 11
M5B - 30
M6B - 4
M9B - 0
M1C - 1
M3C - 23
M4C - 6
M5C - 30
M6C - 4
M9C - 8
M1E - 8
M4E - 4
M5E - 30
M6E - 4
M1G - 4
M4G - 30
M5G - 30
M6G - 17
M1H - 9
M2H - 5
M3H - 23
M4H - 17
M5H - 30
M6H - 15
M1J - 1
M2J - 30
M3J - 6
M4J - 4
M5J - 30
M6J - 30
M1K - 5
M2K - 4
M3K - 2
M4K - 30
M5K - 30
M6K - 26
M1L - 9
M2L - 0
M3L - 5
M4L - 20
M5L - 30
M6L - 4
M9L - 1
M1M - 2
M2M - 1
M3M - 3
M4M - 30
M5M - 23
M6M - 4
M9M - 1
M1N - 4
M2N - 30
M3N - 5
M4N - 4
M5N - 2
M6N - 4
M9N - 1
M1P - 5
M2P - 3
M4P - 10
M5P - 4
M6P - 24
M9P - 8
M1R - 7
M2R - 6
M4R - 17
M5R - 20
M6R - 14
M7R - 13
M9R - 3
M1S - 4
M4S - 30
M5S - 30
M6S - 30
M1T - 14
M4T - 4
M5T - 30
M1V - 3
M4V - 17
M5V - 17
M8V - 12
M9V - 9
M1W - 15
M4W - 4
M5W - 30
M8W - 6
M9W - 4
M1X - 0
M4X - 30
M5X - 30
M8X - 3
M4Y - 30
M7Y - 18
M8Y - 2
M8Z - 14


<h4>
As you can see below, now we have a list of all venues in each postal code area and its categories.
</h4>

In [45]:
nearby_venues.head(10)

,PostalCode,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M3A,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,M3A,43.753259,-79.329656,Sun Life,43.754760,-79.332783,Construction & Landscaping
2,M3A,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,M4A,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,M4A,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant
5,M4A,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
6,M4A,43.725882,-79.315572,The Frig,43.727051,-79.317418,French Restaurant
7,M4A,43.725882,-79.315572,Pizza Nova,43.725824,-79.312860,Pizza Place
8,M5A,43.654260,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
9,M5A,43.654260,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop


<h1>3 - Data analysis</h1>

To start our study, we need to classify each region based on its venues categories. To do that, we will use k-means algoritm.

In [48]:
# one hot encoding
onehot = pd.get_dummies(nearby_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
onehot['PostalCode'] = nearby_venues['PostalCode'] 

# move PostalCode column to the first column
fixed_columns = [onehot.columns[-1]] + list(onehot.columns[:-1])
onehot = onehot[fixed_columns]

onehot.head()

,PostalCode,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,M3A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M3A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M3A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M4A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M4A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [49]:
onehot.shape

(1327, 234)

In [50]:
grouped = onehot.groupby('PostalCode').mean().reset_index()
grouped.head()

,PostalCode,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,M1B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,M1C,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,M1E,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,M1G,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,M1H,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [51]:
grouped.shape

(99, 234)

In [52]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [53]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['PostalCode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
postalcode_venues_sorted = pd.DataFrame(columns=columns)
postalcode_venues_sorted['PostalCode'] = grouped['PostalCode']

for ind in np.arange(grouped.shape[0]):
    postalcode_venues_sorted.iloc[ind, 1:] = return_most_common_venues(grouped.iloc[ind, :], num_top_venues)

postalcode_venues_sorted.head()

,PostalCode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Fast Food Restaurant,Yoga Studio,Deli / Bodega,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store
1,M1C,Bar,Yoga Studio,Department Store,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center
2,M1E,Electronics Store,Breakfast Spot,Restaurant,Rental Car Location,Medical Center,Mexican Restaurant,Intersection,Bank,Yoga Studio,Diner
3,M1G,Coffee Shop,Pharmacy,Korean Restaurant,Deli / Bodega,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center
4,M1H,Lounge,Bank,Bakery,Athletics & Sports,Caribbean Restaurant,Thai Restaurant,Gas Station,Hakka Restaurant,Fried Chicken Joint,Donut Shop


In [54]:
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

grouped_clustering = pd.merge(grouped, dfPostalCodesUniversities[['PostalCode']], how='left', on='PostalCode').drop('PostalCode', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 4, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [55]:
postalcode_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

merged = dfPostalCodesUniversities

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
merged = merged.join(postalcode_venues_sorted.set_index('PostalCode'), on='PostalCode')

merged.head() # check the last columns!

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Universities,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,0.0,0.0,Park,Construction & Landscaping,Food & Drink Shop,Yoga Studio,Department Store,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run
1,M4A,North York,Victoria Village,43.725882,-79.315572,0.0,0.0,Pizza Place,Coffee Shop,Hockey Arena,Portuguese Restaurant,French Restaurant,Donut Shop,Dog Run,Drugstore,Dance Studio,Discount Store
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0.0,0.0,Coffee Shop,Park,Café,Bakery,Breakfast Spot,Theater,Yoga Studio,French Restaurant,Distribution Center,Spa
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,0.0,0.0,Clothing Store,Accessories Store,Vietnamese Restaurant,Boutique,Event Space,Furniture / Home Store,Coffee Shop,Concert Hall,Dessert Shop,Electronics Store
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,1.0,0.0,Coffee Shop,Yoga Studio,Fried Chicken Joint,Sandwich Place,Portuguese Restaurant,Persian Restaurant,Park,Mexican Restaurant,Japanese Restaurant,Italian Restaurant


In [56]:
merged[merged['Universities'] > 0.0]

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Universities,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,1.0,0.0,Coffee Shop,Yoga Studio,Fried Chicken Joint,Sandwich Place,Portuguese Restaurant,Persian Restaurant,Park,Mexican Restaurant,Japanese Restaurant,Italian Restaurant
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,1.0,0.0,Café,Clothing Store,Theater,Coffee Shop,Burrito Place,Ramen Restaurant,Fast Food Restaurant,Sporting Goods Shop,Japanese Restaurant,College Rec Center
22,M1G,Scarborough,Woburn,43.770992,-79.216917,1.0,0.0,Coffee Shop,Pharmacy,Korean Restaurant,Deli / Bodega,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center
27,M2H,North York,Hillcrest Village,43.803762,-79.363452,1.0,0.0,Athletics & Sports,Golf Course,Pool,Mediterranean Restaurant,Dog Run,Dance Studio,Eastern European Restaurant,Drugstore,Donut Shop,Distribution Center
33,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,1.0,0.0,Coffee Shop,Clothing Store,Juice Bar,Restaurant,Bank,Food Court,Movie Theater,Chocolate Shop,Cosmetics Shop,Department Store
34,M3J,North York,"Northwood Park, York University",43.767980,-79.487262,1.0,0.0,Massage Studio,Coffee Shop,Bar,Metro Station,Caribbean Restaurant,Miscellaneous Shop,Yoga Studio,Dessert Shop,Eastern European Restaurant,Drugstore
52,M2M,North York,"Willowdale, Newtonbrook",43.789053,-79.408493,1.0,1.0,Park,Yoga Studio,Deli / Bodega,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center
67,M4P,Central Toronto,Davisville North,43.712751,-79.390197,3.0,0.0,Gym,Breakfast Spot,Gym / Fitness Center,Hotel,Food & Drink Shop,Department Store,Dance Studio,Park,Sandwich Place,Pizza Place
79,M4S,Central Toronto,Davisville,43.704324,-79.388790,1.0,0.0,Dessert Shop,Gym,Sandwich Place,Coffee Shop,Italian Restaurant,Café,Pizza Place,Sushi Restaurant,Thai Restaurant,Seafood Restaurant
80,M5S,Downtown Toronto,"University of Toronto, Harbord",43.662696,-79.400049,7.0,0.0,Café,Sandwich Place,Bookstore,Japanese Restaurant,Restaurant,Bar,Bakery,Yoga Studio,French Restaurant,Comfort Food Restaurant


In [57]:
merged.dropna(subset=['Cluster Labels'], axis=0, inplace=True)
merged['Cluster Labels'] = merged['Cluster Labels'].astype(int)
merged.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Universities,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,0.0,0,Park,Construction & Landscaping,Food & Drink Shop,Yoga Studio,Department Store,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run
1,M4A,North York,Victoria Village,43.725882,-79.315572,0.0,0,Pizza Place,Coffee Shop,Hockey Arena,Portuguese Restaurant,French Restaurant,Donut Shop,Dog Run,Drugstore,Dance Studio,Discount Store
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0.0,0,Coffee Shop,Park,Café,Bakery,Breakfast Spot,Theater,Yoga Studio,French Restaurant,Distribution Center,Spa
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,0.0,0,Clothing Store,Accessories Store,Vietnamese Restaurant,Boutique,Event Space,Furniture / Home Store,Coffee Shop,Concert Hall,Dessert Shop,Electronics Store
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,1.0,0,Coffee Shop,Yoga Studio,Fried Chicken Joint,Sandwich Place,Portuguese Restaurant,Persian Restaurant,Park,Mexican Restaurant,Japanese Restaurant,Italian Restaurant


In [58]:
import matplotlib.cm as cm
import matplotlib.colors as colors# create map
map_clusters = folium.Map(location=[latToronto, lonToronto], zoom_start=zoom)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(merged['Latitude'], merged['Longitude'], merged['PostalCode'], merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<h4>Number of universities and regions in each category:</h4>

In [42]:
merged[['Cluster Labels', 'Universities', 'PostalCode']].groupby(['Cluster Labels', 'Universities']).count().rename(columns={'PostalCode': 'Count'})

,,Count
Cluster Labels,Universities,
0,0.0,12
1,7.0,1
2,1.0,12
3,0.0,73
4,3.0,1


<h4>List of categories and number of regions in each category based on the number of universities:</h4>

In [43]:
merged[['Cluster Labels', 'Universities', 'PostalCode']].groupby(['Universities', 'Cluster Labels']).count().rename(columns={'PostalCode': 'Count'})

Count
Universities Cluster Labels       
0.0          0                  12
             3                  73
1.0          2                  12
3.0          4                   1
7.0          1                   1

<h1>4 - Conclusion</h1>

Based on proposed analysis shown above, there is no apparent relation between the number of universities and venues profiles in each postal code region.